# Manejo de datos faltantes

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos. En particular, a muchos conjuntos de datos interesantes les faltará cierta cantidad de datos. Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar datos faltantes de diferentes maneras. 

discutiremos cómo Pandas elige representarlos y demostraremos algunas herramientas integradas de Pandas para manejar los datos faltantes en Python. 

Nos referiremos a los datos que faltan en general como nulos , NaN o NA valores 

## Compensaciones en convenciones de datos faltantes

Hay una serie de esquemas que se han desarrollado para indicar la presencia de datos faltantes en una tabla o DataFrame. Por lo general, giran en torno a una de dos estrategias: usar una *mask* que indica globalmente los valores que faltan o elegir un *sentinel value* que indica una entrada que falta. 

En el enfoque de enmascaramiento, la máscara puede ser una matriz booleana completamente separada o puede implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor. 

En el enfoque centinela, el valor centinela podría ser alguna convención específica de datos, como indicar un valor entero faltante con -9999 o algún patrón de bits raro, o podría ser una convención más global, como indicar un valor de punto flotante faltante. con NaN (No es un número), un valor especial que forma parte de la especificación de punto flotante IEEE. 

Ninguno de estos enfoques está exento de compensaciones: el uso de una matriz de máscara separada requiere la asignación de una matriz booleana adicional, lo que agrega una sobrecarga tanto en el almacenamiento como en el cálculo. Un valor centinela reduce el rango de valores válidos que se pueden representar y puede requerir una lógica adicional (a menudo no optimizada) en la aritmética de CPU y GPU. Los valores especiales comunes como NaN no están disponibles para todos los tipos de datos. 

Como en la mayoría de los casos en los que no existe una opción universalmente óptima, diferentes lenguajes y sistemas usan diferentes convenciones. Por ejemplo, el lenguaje R usa patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican datos faltantes, mientras que el sistema SciDB usa un byte adicional adjunto a cada celda que indica un estado NA. 

## Datos faltantes en Pandas ¶ 

La forma en que Pandas maneja los valores faltantes está restringida por su dependencia del paquete NumPy, que no tiene una noción integrada de valores NA para tipos de datos que no son de punto flotante. 

Pandas podría haber seguido el ejemplo de R al especificar patrones de bits para cada tipo de datos individual para indicar la nulidad, pero este enfoque resulta ser bastante difícil de manejar. Mientras que R contiene cuatro tipos de datos básicos, NumPy admite mucho más que esto: por ejemplo, mientras que R tiene un solo tipo de entero, NumPy admite catorce tipos de enteros básicos una vez que tenga en cuenta las precisiones disponibles, el signo y el endian de la codificación. Reservar un patrón de bits específico en todos los tipos de NumPy disponibles daría lugar a una cantidad de sobrecarga difícil de manejar en varias operaciones de carcasa especial para varios tipos, que probablemente incluso requieran una nueva bifurcación del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para usarlo como máscara reducirá significativamente el rango de valores que puede representar. 

NumPy admite matrices enmascaradas, es decir, matrices que tienen una matriz de máscara booleana independiente adjunta para marcar los datos como "buenos" o "malos". Pandas podría haberse derivado de esto, pero la sobrecarga tanto en el almacenamiento como en el cálculo y el mantenimiento del código hace que sea una opción poco atractiva. 

Con estas restricciones en mente, Pandas optó por utilizar centinelas para los datos faltantes y, además, optó por utilizar dos valores nulos de Python ya existentes: el punto flotante especial valor ``NaN``, y en Python objeto ``None``. Esta elección tiene algunos efectos secundarios, como veremos, pero en la práctica termina siendo un buen compromiso en la mayoría de los casos de interés. 

### ``None``: Faltan datos pitónicos

El primer valor centinela utilizado por Pandas es ``None``, un objeto singleton de Python que a menudo se usa para datos faltantes en el código de Python. Debido a que es un objeto de Python, ``None`` no se puede usar en ninguna matriz NumPy/Pandas arbitraria, sino solo en matrices con tipo de datos ``'object'``(es decir, matrices de objetos de Python): 

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

Esta ``dtype=object`` significa que la mejor representación de tipo común que NumPy podría inferir para el contenido de la matriz es que son objetos de Python. Si bien este tipo de matriz de objetos es útil para algunos propósitos, cualquier operación en los datos se realizará en el nivel de Python, con mucha más sobrecarga que las operaciones típicamente rápidas que se ven para las matrices con tipos nativos:

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
102 ms ± 3.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
4.13 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



El uso de objetos de Python en una matriz también significa que si realiza agregaciones como ``sum()`` o ``min()`` a través de una matriz con un valor ``None``, por lo general obtendrá un error:

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

Esto refleja el hecho de que la suma entre un número entero y Nonees indefinido.

### ``NaN``: Faltan datos numéricos

La otra representación de datos faltantes, ``NaN``(acrónimo de Not a Number ), es diferente; es un valor de punto flotante especial reconocido por todos los sistemas que usan la representación de punto flotante estándar IEEE: 

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Tenga en cuenta que NumPy eligió un tipo de punto flotante nativo para esta matriz: esto significa que, a diferencia de la matriz de objetos anterior, esta matriz admite operaciones rápidas insertadas en el código compilado. Debes ser consciente de que ``NaN`` es un poco como un virus de datos: infecta cualquier otro objeto que toca. Independientemente de la operación, el resultado de la aritmética con ``NaN`` será otro ``NaN``:

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

Tenga en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no dan como resultado un error) pero no siempre son útiles:

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

### ¡NumPy proporciona algunas agregaciones especiales que ignorarán estos valores faltantes!

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Manten eso en mente ``NaN`` es específicamente un valor de punto flotante; no existe un valor NaN equivalente para números enteros, cadenas u otros tipos.

### NaN y Ninguno en Pandas

``NaN`` y ``None`` ambos tienen su lugar, y Pandas está diseñado para manejarlos casi indistintamente, convirtiendo entre ellos cuando corresponda:

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Para los tipos que no tienen un valor centinela disponible, Pandas convierte automáticamente cuando los valores NA están presentes. Por ejemplo, si establecemos un valor en una matriz de enteros para ``np.nan``, se convertirá automáticamente a un tipo de punto flotante para acomodar el NA:

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Tenga en cuenta que, además de convertir la matriz de enteros en punto flotante, Pandas convierte automáticamente el ``None`` a un valor ``NaN``.

Si bien este tipo de magia puede parecer un poco pirateado en comparación con el enfoque más unificado de los valores de NA en lenguajes específicos de dominio como R, el enfoque de centinela/casting de Pandas funciona bastante bien en la práctica y, según mi experiencia, rara vez causa problemas.

La siguiente tabla enumera las convenciones de upcasting en Pandas cuando se introducen valores NA: 

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Tenga en cuenta que en Pandas, los datos de string siempre se almacenan con un ``object`` dtype.

## Operando con valores nulos

Pandas trata ``None`` y ``NaN`` como esencialmente intercambiables para indicar valores faltantes o nulos. Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas. Ellos son: 

- ``isnull()``: Genera una máscara booleana que indica valores faltantes
- ``notnull()``: Opuesto de ``isnull()``
- ``dropna()``: Devuelve una versión filtrada de los datos
- ``fillna()``: Devolver una copia de los datos con los valores faltantes completados o imputados

Concluiremos con una breve exploración y demostración de estas rutinas. 

### Detectando valores nulos

Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull()`` y ``notnull()``. Cualquiera de los dos devolverá una máscara booleana sobre los datos. 

Por ejemplo:

In [26]:
data = pd.Series([1, np.nan, 'hello', None,'world'])

In [27]:
#data.isnull().sum() Contar null p NaN
data.isnull()

0    False
1     True
2    False
3     True
4    False
dtype: bool

Las máscaras booleanas se pueden usar directamente en una ``Series`` o ``DataFrame`` índice:

In [28]:
#mostrar no nulos 
data[data.notnull()]

0        1
2    hello
4    world
dtype: object

Los métodos ``isnull()`` y ``notnull()`` producen resultados booleanos similares para ``DataFrames``.

### Eliminar valores nulos

Además del enmascaramiento utilizado anteriormente, existen los métodos de conveniencia, ``dropna()`` (que elimina los valores de NA) y ``fillna()``(que rellena los valores de NA). Para ``Series``, el resultado es sencillo:

In [29]:
data.dropna()

0        1
2    hello
4    world
dtype: object

Para ``DataFrame``, hay más opciones. Considera lo siguiente ``DataFrame``:

In [30]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


No podemos descartar valores individuales de un ``DataFrame``; solo podemos soltar filas completas o columnas completas. Dependiendo de la aplicación, es posible que desee uno u otro, por lo que ``dropna()`` da una serie de opciones para un ``DataFrame``.

Por defecto, ``dropna()`` eliminará todas las filas en las que cualquier valor nulo:

In [31]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternativamente, puede soltar los valores NA a lo largo de un eje diferente; ``axis=1`` elimina todas las columnas que contienen un valor nulo:

In [34]:
#df.dropna(axis=1)
df.dropna(axis='columns')

,2
0,2
1,5
2,6


Pero esto también arroja algunos buenos datos; es posible que le interese colocar filas o columnas con todos los valores NA, o la mayoría de los valores NA. Esto se puede especificar a través de los parámetros ``how`` o ``thresh``, que permiten un control preciso del número de valores nulos que se permiten pasar.

El valor predeterminado es ``how='any'``, tal que cualquier fila o columna (dependiendo del ``axis`` palabra clave) que contenga un valor nulo se eliminará. También puede especificar ``how='all'``, que solo eliminará filas/columnas que son todos valores nulos:

In [35]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [36]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Para un control más detallado,El parámetro ``thresh`` le permite especificar un número mínimo de valores no nulos para que se mantenga la fila/columna:

In [39]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


Aquí se eliminaron la primera y la última fila porque contienen solo dos valores no nulos.

### Llenar valores nulos

A veces, en lugar de descartar los valores NA, prefiere reemplazarlos con un valor válido. Este valor puede ser un solo número como cero, o puede ser algún tipo de imputación o interpolación de los buenos valores. Puede hacer esto en el lugar usando el método ``isnull()`` como una máscara, pero debido a que es una operación tan común, Pandas proporciona el método ``fillna()``, que devuelve una copia de la matriz con los valores nulos reemplazados. 

Considera la siguiente ``Series``:

In [40]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Podemos llenar las entradas de NA con un solo valor, (ex: el 0):

In [42]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Podemos especificar un relleno hacia adelante para propagar el valor anterior hacia adelante:

In [43]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

O podemos especificar un relleno para propagar los siguientes valores hacia atrás:

In [45]:
#Interpolacion escalon hacia atras
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Para ``DataFrame``, las opciones son similares, pero también podemos especificar un ``axis`` a lo largo de la cual se realizan los rellenos:

In [46]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [47]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Tenga en cuenta que si un valor anterior no está disponible durante un llenado hacia adelante, el valor NA permanece.